---
## 🌞 0: Introduction



Aditya-L1 is a solar observatory positioned 1.5 million kilometers from Earth at a place called the Lagrangian point L1, providing a continuous view of the Sun. 

SoLEXS is a spectrometer aboard Aditya-L1 that monitors X-ray emissions from the Sun, every second. This data is valuable for studying solar flares, which are bursts of energy released by the Sun.

#### 🚀 `ADITYA_L1_EPOCH_UNIX` = 1693630800 
#### 🚀 `ADITYA_L1_EPOCH_UTC` = 2023-09-02T06:20:00 UTC

---

## 🌞 1: Set up the environment

This cell imports necessary libraries for data handling, plotting, and interactive features. It sets up the environment for reading solar data and creating visualizations.

1. `pandas` for data manipulation and analysis.

2. `matplotlib.pyplot` for creating plots and figures.

3. `astropy.io.fits` to read FITS-format files containing your lightcurve data.

4. `astropy.time.Time` for converting Unix timestamps into human-readable dates/times.

5. `gzip` and `zipfile` for handling compressed files.

6. `tqdm` and `ipywidgets` for visualization and interactive widgets.

7. `ipydatetime` for interactive date/time selection.

In [5]:
%pip install astropy tqdm ipywidgets jupyter-ui-poll ipydatetime -q

from pathlib import Path
from io import BytesIO
from tqdm.auto import tqdm
import zipfile, gzip, io

import ipywidgets as widgets
from IPython.display import Markdown, display, clear_output

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from astropy.io import fits
from astropy.table import Table
from astropy.time import Time

Note: you may need to restart the kernel to use updated packages.


---
## 🌞 2: Take a closer look at the `.lc` LightCurve file

This cell demonstrates how to inspect a sample lightcurve file, showing its structure and header information to help understand the data format before processing.

In [6]:
data_dir = Path('data')
zips     = sorted(data_dir.glob('AL1_SLX_L1_*_v1.0.zip'))
if not zips:
    raise FileNotFoundError(f"No Aditya-L1 ZIPs in {data_dir}")
latest   = zips[-1]

print(f"Most recent data file: {latest.name}")

# Extract and open FITS in-memory
with zipfile.ZipFile(latest, 'r') as z:
    member = next(m for m in z.namelist() if '/SDD2/' in m and m.endswith('.lc.gz'))
    raw    = z.read(member)

with gzip.GzipFile(fileobj=io.BytesIO(raw)) as dec, fits.open(dec) as hdul:

    # Build the “hdul.info()” table
    info = []
    for idx, hdu in enumerate(hdul):
        hdr      = hdu.header
        name     = hdr.get('EXTNAME', 'PRIMARY').strip()
        ver      = hdr.get('EXTVER', 1)
        hdu_type = type(hdu).__name__.replace('HDU','HDU')
        cards    = len(hdr)
        if hdu.data is not None:
            nrows   = hdr.get('NAXIS2','')
            nfields = hdr.get('TFIELDS','')
            dims    = f"{nrows}R x {nfields}C"
            tforms  = [hdr.get(f"TFORM{i}",'') for i in range(1, int(nfields)+1)]
            fmt     = f"[{', '.join(tforms)}]"
        else:
            dims, fmt = '', ''
        info.append({
            "No.":        idx,
            "Name":       name,
            "Ver":        ver,
            "Type":       hdu_type,
            "Cards":      cards,
            "Dimensions": dims,
            "Format":     fmt
        })

    df_info = pd.DataFrame(info)
    display(Markdown(df_info.to_markdown(index=False)))

    # Render full headers for HDU0 & HDU1
    for idx, title in ((0, "Primary HDU"), (1, "Table HDU")):
        hdr = hdul[idx].header
        df  = pd.DataFrame([
            {"Keyword": k, "Value": hdr[k], "Comment": hdr.comments[k]}
            for k in hdr.keys()
        ])
        display(Markdown(f"## {title} Header\n" + df.to_markdown(index=False)))

Most recent data file: AL1_SLX_L1_20250331_v1.0.zip


|   No. | Name    |   Ver | Type        |   Cards | Dimensions   | Format   |
|------:|:--------|------:|:------------|--------:|:-------------|:---------|
|     0 | PRIMARY |     1 | PrimaryHDU  |      15 |              |          |
|     1 | RATE    |     1 | BinTableHDU |      39 | 86400R x 2C  | [D, D]   |

## Primary HDU Header
| Keyword   | Value                          | Comment                                        |
|:----------|:-------------------------------|:-----------------------------------------------|
| SIMPLE    | True                           | conforms to FITS standard                      |
| BITPIX    | 8                              | array data type                                |
| NAXIS     | 0                              | number of array dimensions                     |
| EXTEND    | True                           |                                                |
| MISSION   | ADITYA-L1                      | Name of mission/satellite                      |
| TELESCOP  | AL1                            | Name of mission/satellite                      |
| INSTRUME  | SoLEXS                         | Name of Instrument/detector                    |
| ORIGIN    | SoLEXSPOC                      | Source of FITS file                            |
| CREATOR   | solexs_pipeline-1.2            | Creator of file                                |
| FILENAME  | AL1_SOLEXS_20250331_SDD2_L1.lc | Name of file                                   |
| CONTENT   | LIGHT CURVE                    | File content                                   |
| DATE      | 2025-04-08                     | Creation Date                                  |
| OBS_DATE  | 20250331                       |                                                |
| OBS_ID    | N00_0000_000474                |                                                |
| DATASUM   | 0                              | data unit checksum updated 2025-04-08T17:24:36 |

## Table HDU Header
| Keyword   | Value                     | Comment                                         |
|:----------|:--------------------------|:------------------------------------------------|
| XTENSION  | BINTABLE                  | binary table extension                          |
| BITPIX    | 8                         | array data type                                 |
| NAXIS     | 2                         | number of array dimensions                      |
| NAXIS1    | 16                        | length of dimension 1                           |
| NAXIS2    | 86400                     | length of dimension 2                           |
| PCOUNT    | 0                         | number of group parameters                      |
| GCOUNT    | 1                         | number of groups                                |
| TFIELDS   | 2                         | number of table fields                          |
| EXTNAME   | RATE                      | Extension name                                  |
| CONTENT   | LIGHT CURVE               | File content                                    |
| HDUCLASS  | OGIP                      | format conforms to OGIP standard                |
| HDUVERS   | 1.1.0                     | Version of format (OGIP memo CAL/GEN/92-002a)   |
| HDUDOC    | OGIP memos CAL/GEN/92-007 | Documents describing the format                 |
| HDUVERS1  | 1.0.0                     | Obsolete - included for backwards compatibility |
| HDUVERS2  | 1.1.0                     | Obsolete - included for backwards compatibility |
| HDUCLAS1  | LIGHTCURVE                | Extension contains spectral data                |
| HDUCLAS2  | TOTAL                     |                                                 |
| HDUCLAS3  | COUNTS                    |                                                 |
| FILTER    | SDD2                      | Filter used                                     |
| TTYPE1    | TIME                      |                                                 |
| TFORM1    | D                         |                                                 |
| TTYPE2    | COUNTS                    |                                                 |
| TFORM2    | D                         |                                                 |
| CREATOR   | solexs_pipeline-1.2       |                                                 |
| TSTART    | 1743379200.0              |                                                 |
| TSTOP     | 1743465599.0              |                                                 |
| TIMEDEL   | 1                         |                                                 |
| TIMZERO   | 0                         |                                                 |
| MJDREFI   | 40587                     |                                                 |
| MJDREFF   | 0                         |                                                 |
| TIMESYS   | UTC                       |                                                 |
| TIMEREF   | LOCAL                     |                                                 |
| TIMEUNIT  | s                         |                                                 |
| DATE-OBS  | 2025-03-31 00:00:00       |                                                 |
| DATE-END  | 2025-03-31 23:59:59       |                                                 |
| TELESCOP  | AL1                       |                                                 |
| INSTRUME  | SoLEXS                    |                                                 |
| NUMBAND   | 4                         |                                                 |
| DATASUM   | 2065465298                | data unit checksum updated 2025-04-08T17:24:36  |

---
## 🌞 3: Load the SoLEXS data

This cell contains the `load_solexs_data` function, which reads and processes solar data from ZIP files, handles caching, and outputs a DataFrame for analysis.

In [7]:
def load_solexs_data(paths_file='SoLEXS_dataset.paths',
                     pattern='AL1_SLX_L1_*_v?.?.zip',
                     out_parquet='SoLEXS_dataset.parquet') -> pd.DataFrame:
    """Load solar flare data from ZIP files at multiple paths and build a fresh Parquet dataset."""
    parquet_path = Path(out_parquet)
    
    # Read paths from the paths file
    try:
        with open(paths_file, 'r') as f:
            data_paths = [line.strip() for line in f if line.strip()]
        print(f"Found {len(data_paths)} paths in {paths_file}")
    except FileNotFoundError:
        print(f"Paths file {paths_file} not found.")
        # If parquet exists, return it; otherwise return empty DataFrame
        return pd.read_parquet(parquet_path) if parquet_path.exists() else pd.DataFrame()
    
    # Collect all ZIP files from all paths
    all_zips = []
    for path in data_paths:
        path_obj = Path(path)
        if not path_obj.exists():
            print(f"Warning: Path {path} does not exist, skipping")
            continue
            
        path_zips = sorted(path_obj.glob(pattern))
        if path_zips:
            print(f"Found {len(path_zips)} ZIP files in {path}")
            all_zips.extend(path_zips)
        else:
            print(f"No ZIP files matching '{pattern}' found in {path}")
    
    # If no ZIPs found anywhere, return existing parquet if available
    if not all_zips:
        print("No data files found in any of the provided paths.")
        if parquet_path.exists():
            print(f"Using existing dataset from {out_parquet}")
            return pd.read_parquet(parquet_path)
        else:
            return pd.DataFrame()
    
    print(f"Processing {len(all_zips)} ZIP files from {len(data_paths)} paths")
    
    # Process all ZIPs
    all_dfs = []
    for zp in tqdm(all_zips, desc='Loading FITS data', unit=' files'):
        day = zp.stem.split('_')[3]
        internal = f"{zp.stem}/SDD2/AL1_SOLEXS_{day}_SDD2_L1.lc.gz"

        try:
            with zipfile.ZipFile(zp) as zf:
                try:
                    raw = zf.read(internal)
                except KeyError:
                    print(f"Warning: {internal} not found in {zp.name}, skipping")
                    continue
                    
            with gzip.GzipFile(fileobj=io.BytesIO(raw)) as dec:
                table = Table.read(dec, format='fits')

            df = table.to_pandas()
            df['DATE'] = pd.to_datetime(day, format='%Y%m%d')
            df['TIME'] = df['TIME'].astype(int)
            df['COUNTS'] = df['COUNTS'].astype('Int64')
            all_dfs.append(df[['DATE','TIME','COUNTS']])
                
        except Exception as e:
            print(f"Error processing {zp.name}: {e}")

    # If we have data, build the dataset
    if all_dfs:
        # Combine all data frames
        master = pd.concat(all_dfs, ignore_index=True)
        
        # Sort by DATE and TIME
        master = master.sort_values(['DATE', 'TIME']).reset_index(drop=True)
        
        # Save to parquet, overwriting any existing file
        master.to_parquet(out_parquet, index=False)
        print(f"Built fresh dataset with {len(master):,} rows from {len(all_dfs)} files")
        return master
    else:
        print("No data could be processed from the ZIP files.")
        # Return existing parquet if available
        if parquet_path.exists():
            print(f"Using existing dataset from {out_parquet}")
            return pd.read_parquet(parquet_path)
        else:
            return pd.DataFrame()

# Load data, building a fresh dataset if data is available
SoLEXS_df = load_solexs_data()

# Only display info if we have data
if not SoLEXS_df.empty:
    display(SoLEXS_df)
    display(SoLEXS_df.loc[:, ['DATE', 'TIME']].apply(['min', 'max'], axis=0))
    display(SoLEXS_df.describe().loc[:, ['COUNTS']].T.astype(int))
else:
    print("No data available. Please check the paths in SoLEXS_dataset.paths")

Found 5 paths in SoLEXS_dataset.paths
Found 26 ZIP files in data/archive/solexs_2025Apr19T040188866
Found 46 ZIP files in data/archive/solexs_2025May14T043448388
Found 56 ZIP files in data/archive/solexs_2025May14T044128344
Found 58 ZIP files in data/archive/solexs_2025May14T044300021
Found 60 ZIP files in data/archive/solexs_2025May14T044426758
Processing 246 ZIP files from 5 paths


Loading FITS data:   0%|          | 0/246 [00:00<?, ? files/s]

Built fresh dataset with 21,254,400 rows from 246 files


,DATE,TIME,COUNTS
0,2024-07-01,1719792000,31
1,2024-07-01,1719792001,34
2,2024-07-01,1719792002,33
3,2024-07-01,1719792003,42
4,2024-07-01,1719792004,47
...,...,...,...
21254395,2025-03-31,1743465595,75
21254396,2025-03-31,1743465596,88
21254397,2025-03-31,1743465597,78
21254398,2025-03-31,1743465598,79


,DATE,TIME
min,2024-07-01,1719792000
max,2025-03-31,1743465599


,count,mean,min,25%,50%,75%,max,std
COUNTS,19674852,147,0,30,59,118,321993,551


---
## 🌞 4: Setup interactive widgets and define plotting functions

This cell creates interactive date/time selectors and parameters, allowing users to customize the analysis range and sensitivity for solar flare detection.

In [13]:
# These are default values that are loaded when the notebook initialises
start_date = (SoLEXS_df['DATE'].max() - pd.Timedelta(days=15)).strftime('%Y-%m-%d')
start_time = '00:00:00'

end_date   = SoLEXS_df['DATE'].max().strftime('%Y-%m-%d')
end_time   = '23:59:59'

gap = 60*30    # Seconds of inactivity to split events
sigma = 5   # Threshold = median + sigma * std

In [14]:
# Try to use DatePicker widgets
try:
    # Create proper date picker widgets
    date_widget = widgets.DatePicker(
        description='Start date:',
        value=pd.to_datetime(start_date).date()
    )

    end_date_widget = widgets.DatePicker(
        description='End date:',
        value=pd.to_datetime(end_date).date()
    )
except (ImportError, AttributeError, TypeError):
    # Fall back to text widgets if DatePicker isn't available
    date_widget = widgets.Text(
        description='Start date:',
        value=start_date,
        placeholder='YYYY-MM-DD'
    )

    end_date_widget = widgets.Text(
        description='End date:',
        value=end_date,
        placeholder='YYYY-MM-DD'
    )

# Time inputs (as text)
start_time_widget = widgets.Text(
    description='Start time:',
    value=start_time,
    placeholder='HH:MM:SS'
)

end_time_widget = widgets.Text(
    description='End time:',
    value=end_time,
    placeholder='HH:MM:SS'
)

# Add descriptions for parameter widgets
sigma_widget = widgets.IntSlider(
    value=sigma,
    min=1,
    max=10,
    step=1,
    description='Sigma:',
    tooltip='Threshold = median + sigma * std'
)

gap_widget = widgets.IntSlider(
    value=gap,
    min=60,
    max=7200,
    step=60,
    description='Gap (sec):',
    tooltip='Seconds of inactivity to split events'
)

# Add zoom window widget
zoom_window = 20  # Default: 10 minutes
zoom_widget = widgets.IntSlider(
    value=zoom_window,
    min=1,
    max=60,
    step=1,
    description='Zoom (min):',
    tooltip='Minutes to show around flare peak'
)

# Add a Run button
run_button = widgets.Button(
    description='Update Analysis',
    button_style='success',
    icon='play', 
    tooltip='Update parameters and data window',
    layout=widgets.Layout(margin='20px')
)

# Stats output area
stats_output = widgets.Output()

# Layout with styling
controls = widgets.VBox([
    widgets.HTML("<h3>Solar Flare Analysis Parameters</h3>"),
    widgets.HBox([
        widgets.VBox([date_widget, start_time_widget]),
        widgets.VBox([end_date_widget, end_time_widget])
    ]),
    widgets.HBox([
        widgets.VBox([sigma_widget]),
        widgets.VBox([gap_widget])
    ]),
    widgets.VBox([zoom_widget]),
    run_button,
    stats_output  # Add stats output directly below controls
])

# Function to run analysis
def run_analysis(b):
    global df, times, counts, med, std, flares, zoom_window
    
    # Update global zoom_window from widget
    zoom_window = zoom_widget.value
    
    # Get values from widgets
    if hasattr(date_widget.value, 'strftime'):
        start_date_val = date_widget.value.strftime('%Y-%m-%d')
    else:
        start_date_val = date_widget.value
        
    if hasattr(end_date_widget.value, 'strftime'):
        end_date_val = end_date_widget.value.strftime('%Y-%m-%d')
    else:
        end_date_val = end_date_widget.value
    
    start_time_val = start_time_widget.value
    end_time_val = end_time_widget.value
    sigma_val = sigma_widget.value
    gap_val = gap_widget.value
    
    # Clear previous output
    with stats_output:
        clear_output()
        
        # Show current parameters
        print(f"Analysis parameters:\n")
        print(f"Observation START: {start_date_val} {start_time_val}")
        print(f"Observation  END : {end_date_val} {end_time_val}")
        print(f"Sigma: {sigma_val} (flare detection threshold)")
        print(f"Gap: {gap_val} seconds (between separate flares)")
        print(f"Zoom window: {zoom_window} minutes (for flare detail view)")
        
        # Filter by date range
        df = SoLEXS_df[
            (SoLEXS_df['DATE'] >= start_date_val) & 
            (SoLEXS_df['DATE'] <= end_date_val)
        ].copy()
        
        # Convert time strings to seconds
        h, m, s = map(int, start_time_val.split(':'))
        start_index = h * 3600 + m * 60 + s
        
        h, m, s = map(int, end_time_val.split(':'))
        end_index = len(df) - (86400 - h * 3600 + m * 60 + s)
        
        # Apply time filtering
        df = df.iloc[start_index:end_index]
        
        if len(df) == 0:
            print("\nNo data available for the selected range.")
            return
        
        # Display data window
        print("\n\nData Window:")
        display(df.loc[:, ['DATE', 'TIME']].apply(['min', 'max'], axis=0))
        display(df.describe().loc[:, ['COUNTS']].T.astype(int))
        
        times = df['TIME']
        counts = df['COUNTS']
        
        # Find flares
        med, std = np.nanmedian(counts), np.nanstd(counts)
        mask = counts > (med + sigma_val * std)
        spikes = np.unique(times[mask])
        
        flares = []
        if spikes.size == 0:
            print("\nNo flares detected.")
        else:
            groups = [[spikes[0]]]
            for t in spikes[1:]:
                if t - groups[-1][-1] <= gap_val:
                    groups[-1].append(t)
                else:
                    groups.append([t])
            events = [(g[0], g[-1]) for g in groups]
            print(f"\nDetected {len(events)} {'flares' if len(events) > 1 else 'flare'}:\n")
            for i, ev in enumerate(events, 1):
                t0, t1 = Time(ev[0], format='unix'), Time(ev[1], format='unix')
                # midnight of that day:
                mid = Time(t0.iso[:10]+'T00:00:00', format='isot', scale='utc')
                info = {'start_iso': t0.iso[:-4],
                        'end_iso':   t1.iso[:-4],
                        'start_sod': (t0 - mid).sec,
                        'end_sod':   (t1 - mid).sec}
                flares.append(info)
                print(f"🔥 Flare {i}: {info['start_iso']} → {info['end_iso']}")

---
## 🌞 5. Set control variables and go flare hunting

- **Date and Time Selection**: Allows you to specify the start and end dates/times for the data subset. This focuses the analysis on a particular period, making it easier to examine specific solar events.

- **Sigma (σ)**: A threshold for flare detection sensitivity. Higher values detect only stronger flares by requiring a larger deviation from the mean count rate, reducing false positives.

- **Gap**: Defines the minimum time interval (in seconds) between detected flares. It prevents closely spaced fluctuations from being counted as separate flares, ensuring more accurate event identification.

- **Zoom start and end**: Controls the range of the plot for detailed viewing. Set these to focus on a specific subset of the data timeline for in-depth analysis of individual flares.

In [15]:
# Connect the button to the function
run_button.on_click(run_analysis)

# Display the combined widgets and stats
display(controls)

# Run analysis once to show initial results
run_analysis(None)

---
## 🌞 6: Plot light curves and detect solar flares

This cell defines a function to plot light curves and detect solar flares based on user inputs, providing visual insights into the data.

In [11]:
# Light Curve Plot with Update Button
plot_output = widgets.Output()
update_plot_button = widgets.Button(
    description='Update',
    button_style='info',
    icon='refresh'
)

def update_light_curve(b):
    global df, times, counts, med, std, flares
    
    with plot_output:
        clear_output()
        
        if 'df' not in globals() or len(df) == 0:
            print("No data available to run the analysis.")
            return
            
        # Plot the results
        plot_start_time = Time(times.iloc[0], format='unix').to_datetime().strftime('%H:%M:%S')
        plot_end_time = Time(times.iloc[-1], format='unix').to_datetime().strftime('%H:%M:%S')
        
        plt_times = Time(times, format='unix').to_datetime()
        plt.figure(figsize=(30, 18), dpi=300)
        plt.plot(plt_times, counts)
        
        plt.axhline(med, color='green', linestyle='-', linewidth=2,
                    label=f'Median: {med:.0f}')
        plt.axhline(med + sigma_widget.value * std, color='red', linestyle='-', linewidth=2,
                    label=f'{sigma_widget.value}σ Threshold: {med + sigma_widget.value * std:.0f}')
        
        # Annotate flares
        if 'flares' in globals() and flares:
            for i, flare in enumerate(flares, 1):
                t0, t1 = pd.to_datetime(flare['start_iso']), pd.to_datetime(flare['end_iso'])
                win = (plt_times >= t0) & (plt_times <= t1)
                if not win.any():
                    continue
                
                times_win = plt_times[win]
                counts_win = counts.iloc[win.nonzero()[0]]
                rel_idx = counts_win.values.argmax()
                t_max = times_win[rel_idx]
                y_max = counts_win.iloc[rel_idx]
                
                plt.scatter(t_max, y_max, color='magenta', s=60, zorder=5)
                plt.annotate(
                    f"Peak {i}\n{t_max:%H:%M:%S}",
                    xy=(t_max, y_max),
                    xytext=(-40, 0),
                    textcoords='offset points',
                    ha='right',
                    va='center',
                    color='magenta',
                    fontsize=12,
                    arrowprops=dict(
                        arrowstyle='-|>',
                        color='magenta',
                        lw=1
                    )
                )
        
        plt.legend(fontsize=14)
        plt.gcf().autofmt_xdate()
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %d %H:%M'))
        plt.grid(True)
        plt.xlabel("Time", fontsize=14)
        plt.ylabel("X-Ray Count", fontsize=14)
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        
        # Get date values for title
        if hasattr(date_widget.value, 'strftime'):
            start_date_val = date_widget.value.strftime('%Y-%m-%d')
        else:
            start_date_val = date_widget.value
            
        if hasattr(end_date_widget.value, 'strftime'):
            end_date_val = end_date_widget.value.strftime('%Y-%m-%d')
        else:
            end_date_val = end_date_widget.value
            
        plt.title(
            f"SoLEXS X-ray Lightcurve\n\n"
            f"{start_date_val} {plot_start_time} to {end_date_val} {plot_end_time}",
            fontsize=16, fontweight='bold'
        )
        plt.show()

update_plot_button.on_click(update_light_curve)

display(update_plot_button)
display(plot_output)

# Run once to show initial plot
update_light_curve(None)

Button(button_style='info', description='Update', icon='refresh', style=ButtonStyle())

Output()

## 🌞 7: Zoom into the wildest flare

This cell zooms into the largest flare detected in the previous step, providing a detailed view of its characteristics.

In [12]:
# Zoomed Flare Plot with Update Button
zoom_output = widgets.Output()
update_zoom_button = widgets.Button(
    description='Update',
    button_style='info',
    icon='refresh'
)

def update_zoomed_view(b):
    global df, times, counts, flares, zoom_window
    
    with zoom_output:
        clear_output()
        
        if 'df' not in globals() or len(df) == 0:
            print("No data available to run the analysis.")
            return
            
        if 'flares' not in globals() or not flares:
            print("No flares detected in the current data range.")
            return
        
        # Find the strongest flare
        peak_idx = np.nanargmax(df['COUNTS'])
        
        # Ensure indices are within bounds
        start = max(0, peak_idx - zoom_window*30)
        end = min(len(df), peak_idx + zoom_window*30)
        
        if end - start < 10:
            print("Insufficient data for zoomed view.")
            return
            
        time_subset = Time(df['TIME'].iloc[start:end], format='unix').to_datetime()
        
        plt.figure(figsize=(30, 18), dpi=300)
        plt.plot(time_subset, df['COUNTS'].iloc[start:end])
        plt.grid(True)
        plt.xlabel("Time", fontsize=14)
        plt.ylabel("X-Ray Count", fontsize=14)
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        
        date_str = df['DATE'].iloc[peak_idx].strftime('%Y-%m-%d')
        start_str = time_subset[0].strftime('%H:%M')
        end_str = time_subset[-1].strftime('%H:%M')
        
        plt.title(f"Zoomed-in Solar Flare\n\n{date_str} {start_str} to {end_str}", 
                  fontsize=16, fontweight='bold')
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
        plt.show()

update_zoom_button.on_click(update_zoomed_view)

display(update_zoom_button)
display(zoom_output)

# Run once to show initial zoomed view
update_zoomed_view(None)

Button(button_style='info', description='Update', icon='refresh', style=ButtonStyle())

Output()